In [1]:
import sys
!{sys.executable} -m pip install itk-phasesymmetry itk-anisotropicdiffusionlbr plotly

In [1]:
import itk
from itkwidgets import view, line_profile
import ipywidgets as widgets
from ipywidgets import interactive
import numpy as np

In [2]:
color = itk.imread('0/CHESS_FL12_C_160421_215351.941_COLOR-8-BIT.JPG')

In [3]:
viewer = view(color)
viewer

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageRGBUC2; proxy of <Swig Object of type 'it…

In [4]:
thermal = itk.imread('0/CHESS_FL12_C_160421_215351.941_THERM-16BIT.PNG', itk.F)

In [5]:
view(thermal)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF2; proxy of <Swig Object of type 'itkIma…

In [6]:
diffusion_filter = itk.CoherenceEnhancingDiffusionImageFilter.New(thermal)
diffusion_filter.SetEnhancement(3)
diffusion_filter.Update()

viewer = line_profile(diffusion_filter.GetOutput(), interpolation=False)

def smooth(diffusion_time=1.0, lambda_=0.05, noise_scale=0.5, feature_scale=2.0):
    diffusion_filter.SetDiffusionTime(diffusion_time)
    diffusion_filter.SetLambda(lambda_)
    diffusion_filter.SetNoiseScale(noise_scale)
    diffusion_filter.SetFeatureScale(feature_scale)
    diffusion_filter.Update()
    viewer.image = diffusion_filter.GetOutput()
    
smoother_ui = interactive(smooth, 
                          diffusion_time=(0.1, 10, 0.5),
                          lambda_=(0.0001, 0.1, 0.001),
                          noise_scale=(0.2, 5.0, 0.1),
                          feature_scale=(2.0, 6.0, 0.2))
widgets.VBox([viewer, smoother_ui])

In [7]:
print(itk.size(thermal))

itkSize2 ([640, 512])


In [6]:
dimension = thermal.GetImageDimension()

diffusion_filter = itk.CoherenceEnhancingDiffusionImageFilter.New(thermal)
diffusion_filter.SetEnhancement(3)
diffusion_filter.Update()

phase_symmetry_filter = itk.PhaseSymmetryImageFilter.New(diffusion_filter.GetOutput())

orientation_matrix = itk.Array2D[itk.D](2, dimension)
np_array = np.array([1, 0, 0, 1], dtype=np.float64)
vnl_vector = itk.GetVnlVectorFromArray(np_array)
orientation_matrix.copy_in(vnl_vector.data_block())
phase_symmetry_filter.SetOrientations(orientation_matrix)

phase_symmetry_filter.SetSigma(0.25)
phase_symmetry_filter.SetPolarity(0)
phase_symmetry_filter.SetNoiseThreshold(15.0)
print(diffusion_filter.GetDiffusionTime())

1.0


In [9]:
wavelengths = [2, 2, 4, 4, 6, 6, 8, 8, 12, 12, 16, 16]

scales = len(wavelengths) / dimension
wavelength_matrix = itk.Array2D[itk.D](int(scales), dimension)
np_array = np.array(wavelengths, dtype=np.float64)
vnl_vector = itk.GetVnlVectorFromArray(np_array)
wavelength_matrix.copy_in(vnl_vector.data_block())
phase_symmetry_filter.SetWavelengths(wavelength_matrix)

phase_symmetry_filter.Initialize()
phase_symmetry_filter.Update()
flipper = itk.FlipImageFilter.New(phase_symmetry_filter)
flipper.SetFlipAxes([True, True])
flipper.Update()
itk.imwrite(flipper, '0/thermal_phase_symmetry.mha')
rescaler = itk.RescaleIntensityImageFilter[type(flipper.GetOutput()), itk.Image[itk.UC, dimension]].New(phase_symmetry_filter)
itk.imwrite(rescaler, '0/thermal_phase_symmetry.png')
view(flipper, ui_collapsed=True)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF2; proxy of <Swig Object of type 'itkIma…

In [12]:
optical = itk.imread('0/CHESS_FL12_C_160421_215351.941_COLOR-8-BIT.JPG', itk.F)
view(optical, ui_collapsed=True)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF2; proxy of <Swig Object of type 'itkIma…

In [13]:
print(itk.size(thermal))
print(itk.size(optical))

itkSize2 ([640, 512])
itkSize2 ([6576, 4384])


In [14]:
shrink_factors = [10, 10]
optical_shrunk = itk.bin_shrink_image_filter(optical, shrink_factors=shrink_factors)
itk.imwrite(optical_shrunk, '0/optical_shrunk.mha')
view(optical_shrunk)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF2; proxy of <Swig Object of type 'itkIma…

In [15]:
diffusion_filter = itk.CoherenceEnhancingDiffusionImageFilter.New(optical_shrunk)
diffusion_filter.SetEnhancement(3)
diffusion_filter.Update()

viewer = line_profile(diffusion_filter.GetOutput(), interpolation=False)

def smooth(diffusion_time=3.0, lambda_=0.05, noise_scale=0.5, feature_scale=2.0):
    diffusion_filter.SetDiffusionTime(diffusion_time)
    diffusion_filter.SetLambda(lambda_)
    diffusion_filter.SetNoiseScale(noise_scale)
    diffusion_filter.SetFeatureScale(feature_scale)
    diffusion_filter.Update()
    viewer.image = diffusion_filter.GetOutput()
    
smoother_ui = interactive(smooth, 
                          diffusion_time=(0.1, 10, 0.5),
                          lambda_=(0.0001, 0.1, 0.001),
                          noise_scale=(0.2, 5.0, 0.1),
                          feature_scale=(2.0, 6.0, 0.2))
widgets.VBox([viewer, smoother_ui])

In [16]:
dimension = optical_shrunk.GetImageDimension()

diffusion_filter = itk.CoherenceEnhancingDiffusionImageFilter.New(optical_shrunk)
diffusion_filter.SetEnhancement(3)
diffusion_filter.SetDiffusionTime(3.0)
diffusion_filter.Update()
smoothed = diffusion_filter.GetOutput()

padded = itk.fft_pad_image_filter(smoothed)
padded.DisconnectPipeline()
# Bug! todo: fix me
region = padded.GetBufferedRegion()
region.SetIndex([0,0])
padded.SetRegions(region)


phase_symmetry_filter = itk.PhaseSymmetryImageFilter.New(padded)

orientation_matrix = itk.Array2D[itk.D](2, dimension)
np_array = np.array([1, 0, 0, 1], dtype=np.float64)
vnl_vector = itk.GetVnlVectorFromArray(np_array)
orientation_matrix.copy_in(vnl_vector.data_block())
phase_symmetry_filter.SetOrientations(orientation_matrix)

phase_symmetry_filter.SetSigma(0.25)
phase_symmetry_filter.SetPolarity(0)
phase_symmetry_filter.SetNoiseThreshold(30.0)

In [17]:
wavelengths = [2, 2, 4, 4, 6, 6, 8, 8, 12, 12, 16, 16]

scales = len(wavelengths) / dimension
wavelength_matrix = itk.Array2D[itk.D](int(scales), dimension)
np_array = np.array(wavelengths, dtype=np.float64)
vnl_vector = itk.GetVnlVectorFromArray(np_array)
wavelength_matrix.copy_in(vnl_vector.data_block())
phase_symmetry_filter.SetWavelengths(wavelength_matrix)

phase_symmetry_filter.Initialize()
phase_symmetry_filter.UpdateLargestPossibleRegion()

flipper = itk.FlipImageFilter.New(phase_symmetry_filter)
flipper.SetFlipAxes([True, True])
flipper.Update()
itk.imwrite(flipper, '0/optical_phase_symmetry.mha')
rescaler = itk.RescaleIntensityImageFilter[type(flipper.GetOutput()), itk.Image[itk.UC, dimension]].New(phase_symmetry_filter)
itk.imwrite(rescaler, '0/optical_phase_symmetry.png')
view(flipper, ui_collapsed=True)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF2; proxy of <Swig Object of type 'itkIma…